# Set-up

In [1]:
import os
import torch
import numpy as np
import pandas as pd 
import seqdata as sd
import xarray as xr
from eugene import models
from eugene.models import zoo
from eugene import train
from eugene import preprocess as pp

In [9]:
# Model log directory
log_dir = "/cellar/users/aklie/projects/ML4GLand/use_cases/ResidualBind/models/2023_12_16/ResidualBind"

# Load training and validation data

In [2]:
# Load the dataset
sdata_train = sd.open_zarr("/cellar/users/aklie/data/ml4gland/pubs/koo21_gia/log_norm/rnacompete2013_train.zarr")
sdata_train["train_val"] = xr.DataArray([True] * sdata_train.dims["_sequence"], dims=["_sequence"])
sdata_valid = sd.open_zarr("/cellar/users/aklie/data/ml4gland/pubs/koo21_gia/log_norm/rnacompete2013_valid.zarr")
sdata_valid["train_val"] = xr.DataArray([False] * sdata_valid.dims["_sequence"], dims=["_sequence"])
sdata = xr.concat([sdata_train, sdata_valid], dim="_sequence", data_vars="minimal")
sdata["ohe_seq"] = sdata["inputs"]
sdata.load()

<xarray.Dataset>
Dimensions:    (_sequence: 120326, _length: 41, _ohe: 4, _target: 244)
Dimensions without coordinates: _sequence, _length, _ohe, _target
Data variables:
    inputs     (_sequence, _length, _ohe) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    rbp_id     (_target) <U11 'RNCMPT00100' 'RNCMPT00101' ... 'RNCMPT00009'
    rbp_index  (_target) int64 0 1 2 3 4 5 6 7 ... 237 238 239 240 241 242 243
    targets    (_sequence, _target) float32 -0.6122 -0.6356 ... -0.1652 0.984
    train_val  (_sequence) bool True True True True ... False False False False
    ohe_seq    (_sequence, _length, _ohe) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0

# Load model

In [3]:
# Instantiate an architecture
arch = zoo.ResidualBind(
    input_len=41,
    output_dim=1
)

In [4]:
# Initialize the weights
models.init_weights(arch)

In [5]:
# Instantiate a sequence module
module = models.SequenceModule(
    arch=arch,
    task="regression",
    loss_fxn="mse",
    optimizer_lr=0.001,
    scheduler="reduce_lr_on_plateau",
    scheduler_monitor="val_pearson_epoch",
    metric="pearson",
    seed=1234,
)

[rank: 0] Global seed set to 1234


In [6]:
for index in range(sdata.dims["_target"]):
    sdata["target"] = sdata["targets"][:, index]
    rbp_id = sdata["rbp_id"].values[index]
    print(rbp_id)
    break

RNCMPT00100
RNCMPT00101
RNCMPT00102
RNCMPT00103
RNCMPT00104
RNCMPT00105
RNCMPT00106
RNCMPT00107
RNCMPT00108
RNCMPT00109
RNCMPT00010
RNCMPT00110
RNCMPT00111
RNCMPT00112
RNCMPT00113
RNCMPT00114
RNCMPT00116
RNCMPT00117
RNCMPT00118
RNCMPT00119
RNCMPT00011
RNCMPT00120
RNCMPT00121
RNCMPT00122
RNCMPT00123
RNCMPT00124
RNCMPT00126
RNCMPT00127
RNCMPT00129
RNCMPT00012
RNCMPT00131
RNCMPT00132
RNCMPT00133
RNCMPT00134
RNCMPT00136
RNCMPT00137
RNCMPT00138
RNCMPT00139
RNCMPT00013
RNCMPT00140
RNCMPT00141
RNCMPT00142
RNCMPT00143
RNCMPT00144
RNCMPT00145
RNCMPT00146
RNCMPT00147
RNCMPT00148
RNCMPT00149
RNCMPT00014
RNCMPT00150
RNCMPT00151
RNCMPT00152
RNCMPT00153
RNCMPT00154
RNCMPT00155
RNCMPT00156
RNCMPT00157
RNCMPT00158
RNCMPT00159
RNCMPT00015
RNCMPT00160
RNCMPT00161
RNCMPT00162
RNCMPT00163
RNCMPT00164
RNCMPT00165
RNCMPT00166
RNCMPT00167
RNCMPT00168
RNCMPT00169
RNCMPT00016
RNCMPT00170
RNCMPT00171
RNCMPT00172
RNCMPT00173
RNCMPT00174
RNCMPT00175
RNCMPT00176
RNCMPT00177
RNCMPT00178
RNCMPT00179
RNCMPT00017
RNCM

In [7]:
# Grab the training data for this RBP
sdata = sdata.sel(_sequence=sdata["target"].notnull())

In [10]:
# Train the model
trainer = train.fit_sequence_module(
    model=module,
    sdata=sdata,
    seq_var="ohe_seq",
    target_vars=["target"],
    in_memory=True,
    train_var="train_val",
    epochs=100,
    batch_size=100,
    early_stopping_monitor="val_pearson_epoch",
    early_stopping_patience=20,
    early_stopping_mode="max",
    model_checkpoint_monitor="val_pearson_epoch",
    model_checkpoint_k=5,
    model_checkpoint_mode="max",
    log_dir=os.path.join(log_dir, rbp_id),
    transforms={"ohe_seq": lambda x: torch.tensor(x.transpose(0, 2, 1), dtype=torch.float32)},
    name="",
    version="",
    seed=1234,
    return_trainer=True
)

[rank: 0] Global seed set to 1234


Dropping 0 sequences with NaN targets.
Loading ohe_seq and ['target'] into memory


/cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib ...
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0

Sanity Checking: 0it [00:00, ?it/s]

/cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/cellar/users/aklie/opt/miniconda3/envs/ml4gland/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
# Move best model to log directory
print("Copying best model to log directory")
best_model_path = trainer.checkpoint_callback.best_model_path
copy_path = os.path.join(log_dir, rbp_id, "best_model.ckpt")
os.system(f"cp {best_model_path} {copy_path}")

# DONE!

---